In [ ]:
	
%reload_ext sql
%reload_ext mocodo

In [ ]:
%config SqlMagic.dsn_filename = "../connexion_BD.ini" 
# ajouter ou changer ce fichier pour définir sa connexion. Le fichier doit être de la forme
# [pg]
# drivername = postgresql+psycopg
# username = 
# password = 
# host = 
# port = 
# database = 
%sql --section pg
%config SqlMagic.displaylimit = 0 #pas de limite dans le nombre de tuples du résultat

# TP : Implémentation de contraintes

## Important : tout au long du TP, faites des tests de vos contraintes en insérant, supprimant, modifiant des tuples

Un club de danse souhaite se doter d'une base de données pour gérer ses intervenants dans les cours. Après analyse, on propose le schéma Entités/Associations ci-dessous. Les niveaux des cours sont à prendre parmi les valeurs 'Débutant', 'Avancé' ou 'Expert'.

In [ ]:
%%mocodo --select mcd --colors ocean  --output_dir ./ -t arrange

Professeur: num, nom,prenom, telephone
Salarié: date_embauche, echelon, salaire_mensuel
Vacataire: statut
Cours: code, intitulé, niveau
Contrat : Date_contrat, Salaire_horaire

Est_responsable, 0N Professeur, 11 Cours
Intervient, 0N professeur, 1N Cours
EF, _11 Contrat, 1N Vacataire

/XT\ Professeur <- Vacataire, Salarié
(I) -> Intervient, ..Professeur, ..Cours, --Est_responsable

### 1 Traduisez ce schéma en relationnel ; en ne tenant pas compte pour l'instant des contraintes  *XT* et *I*. 

Créez les relations obtenues, ainsi que les clés et clés étrangères qui en découlent.
- indication 1 : Puisque qu'un cours a exactement un responsable, cette association se traduira par une clé étrangère dans la relation "Cours", ne pouvant prendre la valeur NULL.
- indication 2 : Les spécialisations et entités faibles induisent une contrainte d'existence ; par exemple, supprimer un professeur de la base induit de le supprimer également en tant que salarié ou vacataire. De même supprimer un vacataire, induit de supprimer les contrats qui s'y rapportent. Faites en sorte que toutes ces suppressions en cascade soient automatiques (option : ON DELETE CASCADE des clés étrangères).

In [ ]:
#le script de création est dans l'espace WEB accessible aux étudiants. (http://lifbda.pages.univ-lyon1.fr/TP/script_club_de_danse.sql)
with open("../../TP/script_club_de_danse.sql","r") as fichier:
    %sql {{fichier.read()}}

## contraintes

[Documentation](https://docs.postgresql.fr/15/ddl-constraints.html)

Les types de données sont un moyen de restreindre la nature des données qui peuvent être stockées dans une table. Pour beaucoup d'applications, toutefois, la contrainte fournie par ce biais est trop grossière. Par exemple, une colonne qui contient le prix d'un produit ne doit accepter que des valeurs positives. Mais il n'existe pas de type de données natif qui n'accepte que des valeurs positives. Un autre problème peut provenir de la volonté de contraindre les données d'une colonne par rapport aux autres colonnes ou lignes. Par exemple, dans une table contenant des informations de produit, il ne peut y avoir qu'une ligne par numéro de produit.

Pour cela, SQL permet de définir des contraintes sur les colonnes et les tables. Les contraintes donnent autant de contrôle sur les données des tables qu'un utilisateur peut le souhaiter. Si un utilisateur tente de stocker des données dans une colonne en violation d'une contrainte, une erreur est levée. Cela s'applique même si la valeur vient de la définition de la valeur par défaut.

### Contraintes de vérification

La contrainte de vérification est la contrainte la plus générique qui soit. Elle permet d'indiquer que la valeur d'une colonne particulière doit satisfaire une expression booléenne (valeur de vérité). Par exemple, pour obliger les prix des produits à être positifs, on peut utiliser :

```
CREATE TABLE produits (
    no_produit integer,
    nom text,
    prix numeric CHECK (prix > 0)
);
```

### Contraintes de non-nullité (NOT NULL)

Une contrainte NOT NULL indique simplement qu'une colonne ne peut pas prendre la valeur NULL. Par exemple :

```
CREATE TABLE produits (
    no_produit integer NOT NULL,
    nom text NOT NULL,
    prix numeric
);
```

### Contraintes d'unicité

Les contraintes d'unicité garantissent l'unicité des données contenues dans une colonne ou un groupe de colonnes par rapport à toutes les lignes de la table.


```
CREATE TABLE produits (
    no_produit integer UNIQUE,
    nom text,
    prix numeric
);
```

### Clés primaires

Une contrainte de type clé primaire indique qu'une colonne, ou un groupe de colonnes, peuvent être utilisés comme un identifiant unique de ligne pour cette table. Ceci nécessite que les valeurs soient à la fois uniques et non NULL. Les définitions de table suivantes acceptent de ce fait les mêmes données :

```
CREATE TABLE produits (
	no_produit integer UNIQUE NOT NULL,
	nom text,
	prix numeric
);


CREATE TABLE produits (
	no_produit integer PRIMARY KEY,
	nom text,
	prix numeric
);
```

### Clés étrangères

Une contrainte de clé étrangère stipule que les valeurs d'une colonne (ou d'un groupe de colonnes) doivent correspondre aux valeurs qui apparaissent dans les lignes d'une autre table. On dit que cela maintient l'intégrité référentielle entre les deux tables.

Soit la table de produits, déjà utilisée plusieurs fois :

```
CREATE TABLE produits (
	no_produit integer PRIMARY KEY,
	nom text,
	prix numeric
);
```

Soit également une table qui stocke les commandes de ces produits. Il est intéressant de s'assurer que la table des commandes ne contient que des commandes de produits qui existent réellement. Pour cela, une contrainte de clé étrangère est définie dans la table des commandes qui référence la table « produits » :

```
CREATE TABLE commandes (
	id_commande integer PRIMARY KEY,
	no_produit integer REFERENCES produits (no_produit),
	quantite integer
);
```

Une clé étrangère peut aussi contraindre et référencer un groupe de colonnes. Comme cela a déjà été évoqué, il faut alors l'écrire sous forme d'une contrainte de table. Exemple de syntaxe :

```
CREATE TABLE t1 (
      a integer PRIMARY KEY,
      b integer,
      c integer,
      FOREIGN KEY (b, c) REFERENCES autre_table (c1, c2)
);
```

### 2 La contrainte  *I* sur le schéma traduit le fait que si un professeur est responsable d'un cours, alors il intervient dans ce cours. C'est une implication/inclusion : tout couple (professeur, cours) de l'association "Responsable" doit exister dans l'association "Intervient".

- A. Écrivez la dépendance d'inclusion qui traduit cette contrainte dans la base de données
- B. Cette dépendance d'inclusion peut-elle être implantée par une clé étrangère dans la relation 'cours' ?
- C. Créez cette contrainte dans la base de données.
- D. Créez un professeur "Jean Wiejacka" (sans se soucier encore qu'il soit vacataire ou salarié). Créez un cours de salsa débutant, pour lequel Jean Wiejacka est le responsable. Quel problème rencontrez vous ?

#### EXPLICATION 

pour ajouter une nouvelle contrainte : 

```
alter table <table_name> add constraint ...
```

#### 2 A : Écrivez la dépendance d'inclusion qui traduit cette contrainte dans la base de données

Réponse : 

#### 2 B : Cette dépendance d'inclusion peut-elle être implantée par une clé étrangère dans la relation 'cours' ?

Réponse : 

In [ ]:
-- 2 C
%%sql

-- compléter

In [ ]:
-- 2 D
%%sql

-- création de Jean Wiejacka professeur de salsa
insert into professeurs values (1,'Wiejacka','Jean','0638103978');

In [ ]:
-- 2 D
%%sql
-- tentative de création du cours de SALSA, qui doit obligatoirement avoir un responsable
insert into cours values(1,'Salsa',1, 'Débutant'); -- Jean est le responsable de la salsa.

#### 2 D : Quel problème rencontrez vous ?

Réponse : 

### Une transaction est un ensemble de mises à jour considérées comme une seule opération, sans limite sur le nombre d'opérations dans une même transaction. Les SGBD relationnels garantissent que les contraintes sont satisfaites AVANT et APRES la transaction. Ce qui se passe PENDANT la transaction dépend des capacités des SGBD et des choix de l'opérateur. La vérification de certaines contraintes, comme les clés étrangères sous PostgreSQL, peut être différée à l'issue de la transaction.

- A. Modifiez, lorsque cela est nécessaire, les déclarations de clés étrangères afin de différer leur vérification en fin de transaction.
- B. Procédez maintenant à la création du cours de SALSA à l'aide d'une transaction contenant plusieurs insertions de tuples.

## contraintes différées

la contrainte peut être déférée ou ne peut pas l’être :

```
ALTER TABLE ... ADD CONSTRAINT ... NOT DEFERRABLE => n''est jamais déférable
ALTER TABLE ... ADD CONSTRAINT ... DEFERRABLE => est déferrable (mais uniquement sur demande, pas par défaut, donc INITIALY IMMEDIATE)
ALTER TABLE ... ADD CONSTRAINT ... DEFERRABLE INITIALY IMMEDIATE => (même que précédemment : est déferrable, mais uniquement sur demande, pas par défaut)
ALTER TABLE ... ADD CONSTRAINT ... DEFERRABLE INITIALY DEFERRED => est déferrable et systématiquement déférée, sauf si on lui impose le contraire à l'aide du SET CONSTRAINTS...
```


In [ ]:
%%sql

-- Pour casser le cycle, on permet de différer la clé étrangère qu'on a créée sur "Cours" :
-- compléter

## Transaction

[Documentation](https://docs.postgresql.fr/15/tutorial-transactions.html)

Les transactions sont un concept fondamental de tous les systèmes de bases de données. Une transaction assemble plusieurs étapes en une seule opération tout ou rien. Les états intermédiaires entre les étapes ne sont pas visibles par les transactions concurrentes. De plus, si un échec survient qui empêche le succès de la transaction, alors aucune des étapes n'affecte la base de données.

Si l'on considère, par exemple, la base de données d'une banque qui contient le solde de différents comptes clients et le solde total des dépôts par branches et que l'on veut enregistrer un virement de 100 euros du compte d'Alice vers celui de Bob, les commandes SQL peuvent ressembler à cela (après simplification) :

```
UPDATE comptes SET balance = balance - 100.00
    WHERE nom = 'Alice';
UPDATE branches SET balance = balance - 100.00
    WHERE nom = (SELECT nom_branche FROM comptes WHERE nom = 'Alice');
UPDATE comptes SET balance = balance + 100.00
    WHERE nom = 'Bob';
UPDATE branches SET balance = balance + 100.00
    WHERE nom = (SELECT nom_branche FROM comptes WHERE nom = 'Bob');

```

Ce ne sont pas les détails des commandes qui importent ici ; le point important est la nécessité de plusieurs mises à jour séparées pour accomplir cette opération assez simple. Les employés de la banque veulent être assurés que, soit toutes les commandes sont effectuées, soit aucune ne l'est. Il n'est pas envisageable que, suite à une erreur du système, Bob reçoive 100 euros qui n'ont pas été débités du compte d'Alice. De la même façon, Alice ne restera pas longtemps une cliente fidèle si elle est débitée du montant sans que celui-ci ne soit crédité sur le compte de Bob. Il est important de garantir que si quelque chose se passe mal, aucune des étapes déjà exécutées n'est prise en compte. Le regroupement des mises à jour au sein d'une transaction apporte cette garantie. Une transaction est dite atomique : du point de vue des autres transactions, elle passe complètement ou pas du tout.

Il est également nécessaire de garantir qu'une fois la transaction terminée et validée par la base de données, les transactions sont enregistrées définitivement et ne peuvent être perdues, même si une panne survient peu après. Ainsi, si un retrait d'argent est effectué par Bob, il ne faut absolument pas que le débit de son compte disparaisse suite à une panne survenant juste après son départ de la banque. Une base de données transactionnelle garantit que toutes les mises à jour faites lors d'une transaction sont stockées de manière persistante (c'est-à-dire sur disque) avant que la transaction ne soit déclarée validée.

Une autre propriété importante des bases de données transactionnelles est en relation étroite avec la notion de mises à jour atomiques : quand plusieurs transactions sont lancées en parallèle, aucune d'entre elles ne doit être capable de voir les modifications incomplètes effectuées par les autres. Ainsi, si une transaction calcule le total de toutes les branches, inclure le débit de la branche d'Alice sans le crédit de la branche de Bob, ou vice-versa, est une véritable erreur. Les transactions doivent donc être tout ou rien, non seulement pour leur effet persistant sur la base de données, mais aussi pour leur visibilité au moment de leur exécution. Les mises à jour faites jusque-là par une transaction ouverte sont invisibles aux autres transactions jusqu'à la fin de celle-ci. À ce moment, toutes les mises à jour deviennent simultanément visibles.

Sous PostgreSQL, une transaction est déclarée en entourant les commandes SQL de la transaction par les commandes BEGIN et COMMIT. La transaction bancaire ressemble alors à ceci :

```
BEGIN;
UPDATE comptes SET balance = balance - 100.00
    WHERE nom = 'Alice';
-- etc etc
COMMIT;
```

l est possible d'augmenter la granularité du contrôle des instructions au sein d'une transaction en utilisant des points de retournement (savepoint). Ceux-ci permettent d'annuler des parties de la transaction tout en validant le reste. Après avoir défini un point de retournement à l'aide de SAVEPOINT, les instructions exécutées depuis ce point peuvent, au besoin, être annulées avec ROLLBACK TO. Toutes les modifications de la base de données effectuées par la transaction entre le moment où le point de retournement a été défini et celui où l'annulation est demandée sont annulées, mais les modifications antérieures à ce point sont conservées.

Le retour à un point de retournement ne l'annule pas. Il reste défini et peut donc être utilisé plusieurs fois. À l'inverse, lorsqu'il n'est plus nécessaire de revenir à un point de retournement particulier, il peut être relâché, ce qui permet de libérer des ressources système. Il faut savoir toutefois que relâcher un point de retournement ou y revenir relâche tous les points de retournement qui ont été définis après.

Tout ceci survient à l'intérieur du bloc de transaction, et n'est donc pas visible par les autres sessions en cours sur la base de données. Si le bloc est validé, et à ce moment-là seulement, toutes les actions validées deviennent immédiatement visibles par les autres sessions, tandis que les actions annulées ne le seront jamais.

Reconsidérant la base de données de la banque, on peut supposer vouloir débiter le compte d'Alice de $100.00, somme à créditer sur le compte de Bob, mais considérer plus tard que c'est le compte de Wally qu'il convient de créditer. À l'aide des points de retournement, cela peut se dérouler ainsi :

```
BEGIN;
UPDATE comptes SET balance = balance - 100.00
    WHERE nom = 'Alice';
SAVEPOINT mon_pointdesauvegarde;
UPDATE comptes SET balance = balance + 100.00
    WHERE nom = 'Bob';
-- oups ... oublions ça et créditons le compte de Wally
ROLLBACK TO mon_pointdesauvegarde;
UPDATE comptes SET balance = balance + 100.00
    WHERE nom = 'Wally';
COMMIT;
```

In [ ]:
%%sql

--On peut maintenant faire l'insertion en deux temps au sein d'une seule et même transaction :
-- compléter


### 4 Le cahier des charges spécifie qu'un cours a obligatoirement (au moins) un intervenant (participation obligatoire à l'association "Intervient"). Peut-on considérer que cette contrainte est bien garantie dans notre base de données ?

(petit conseille : règle de projection dans les DI , pas encore vue mais facile à comprendre intuitivement)

Réponse : 



### Implanter la contrainte  *X*indiquée dans la spécialisation des professeurs, indiquant qu'un professeur ne peut pas être à la fois un salarié et un vacataire.

- Indication : Cette contrainte peut se programmer à l'aide de deux commandes "CHECK" dans la déclaration des relations "salaries" et "vacataires", par exemple sous la forme "Check (est_vacataire(num_prof) = false)" - où "est_vacataire(num_prof)" est une fonction qui retourne vrai si "num_prof" existe déjà dans vacataires, faux sinon.

Voici pour servir d'exemple une proposition de code de création d'une fonction qui renvoie VRAI si le numéro passé en argument est celui d'un vacataire.

```
drop function if exists est_vacataire;
CREATE function est_vacataire(un_num_prof integer) returns boolean
as $$
begin
 	perform num_vacataire  
 	from vacataires 
 	where num_vacataire = un_num_prof;
 
 	return (found);
end;
 $$ language plpgsql;
```

In [ ]:
%%sql

-- création d'une fonction qui teste si un prof est dans les vacataires
-- compléter

In [ ]:
%%sql

-- création d'une fonction qui teste si un prof est dans les salariés
-- compléter

In [ ]:
-- 5
%%sql

-- On peut maintenant ajouter des contraintes CHECK dans les relations salarie et vacataires
-- Pour garantir l'exclusion mutuelle à l'insertion / modification
-- compléter


### 6 Implanter la contrainte *T* de la spécialisation, qui impose que tous les professeurs ont la position de salarié ou de vacataire.

- A. Lors de la création d'un professeur : ajouter une contrainte "check" qui s'assure que ce professeur est bien un salarié ou un vacataire.
- B. Différez les contraintes de clés étrangères dans "salariés" et "vacataires", de façon à pouvoir momentanément créer un salarié ou vacataire qui n'est pas un professeur.
- C. Effectuez alors l'insertion d'un nouveau professeur - vacataire dans une transaction qui crée d'abord le vacataire pour le professeur.

In [ ]:
-- 6 B
%%sql

-- Différez les contraintes de clés étrangères dans "salariés" et "vacataires"
-- compléter


In [ ]:
%%sql
-- Au préalable dans les données saisies on indique que le professeur 1 est salarié pour ne pas violer la contrainte déclarée après
insert into salaries values(1,'2022-07-10',2,1500);

In [ ]:
-- 6 A
%%sql

-- ajouter une contrainte "check" qui s'assure que ce professeur est bien un salarié ou un vacataire.
-- compléter


In [ ]:
-- 6 C
%%sql

-- compléter


### 7 Ce n'est pas tout à fait fini ! Il faut encore s'assurer que personne ne puisse supprimer une ligne des relations "vacataires" ou "salariés" car cela contredirait la contrainte  *T* (un professeur serait ni salarié, ni vacataire). Mais si on fait une contrainte trop forte qui interdit ces suppressions, on ne pourra plus faire les opérations suivantes : modifier un professeur en le passant de vacataire à salarié (ou l'inverse), ou même supprimer un professeur - car Postgre va tenter de supprimer le vacataire/salarié correspondant (en cascade). Voici comment résoudre ces problèmes.

- A. Créer une fonction  *est_prof*qui teste si un numéro de professeur (fourni en entrée) existe parmi les professeurs.
- B. Faire un trigger (https://doc.postgresql.fr/16/sql-createtrigger.html) différable sur "salaries" qui interdit de supprimer un salarié si celui est un professeur qui existe encore et qu'il n'est pas dans "vacataires".
- C. Faire un trigger différable sur "vacataires" qui interdit de supprimer un vacataire si celui est un professeur qui existe encore et qu'il n'est pas dans "salaries".
- D. Vérifiez maintenant que vous pouvez 1) supprimer un professeur et 2) passer un professeur de salarié à vacataire en utilisant une transaction à deux étapes.

Aide : comment faire le trigger sur salariés :

```
-- Il faut d'abord faire une fonction qui sera utilisée par le trigger : elle ne doit pas prendre de paramètres, et retourner un objet de type 'trigger'.
drop function if exists check_prof_vacataire cascade; 
CREATE FUNCTION check_prof_vacataire() RETURNS trigger 
as $$
begin
	if  not est_prof(old.num_salarie) then return null; -- le prof a été supprimé (en cours de suppression)
	elsif est_vacataire(old.num_salarie) then return null; -- le salarié est bien dans vacataires.
	else raise exception 'professeur doit etre vacataire si plus salarie';
	end if;
end;
$$ LANGUAGE plpgsql;

create constraint trigger check_prof_vacataire
	after delete or update on salaries deferrable for each row execute function check_prof_vacataire();
```

In [ ]:
-- 7 A
%%sql
-- Créer une fonction est_prof
-- compléter



In [ ]:
-- 7 B
%%sql

-- Interdiction de supprimer un salarié s'il n'est pas vacataire.
-- Contrainte forcément différable car il faut pouvoir avant l'insérer dans vacataire 
-- compléter

In [ ]:
-- 7 C
%%sql

-- Interdiction de supprimer un vacataire s'il n'est pas salarié.
-- Contrainte forcément différable car il faut pouvoir avant l'insérer dans salaries. 
-- compléter

In [ ]:
-- 7 D 2)
%%sql

-- on transforme le professeur 1 en vacataire étudiant (il était salarié)
-- compléter

In [ ]:
-- 7 D 1)
%%sql

-- on vérifie que la suppression d'un professeur fonctionne toujours
-- compléter


In [ ]:
%%sql

select * 
from professeurs p  
     left join vacataires v on p.num_prof=v.num_vacataire
     left join salaries s on p.num_prof = s.num_salarie;